# Accessing & managing financial data

## Fama-French data

We start by downloading some famous Fama-French factors [Fama1993](https://doi.org/2329112.) and portfolio returns commonly used in empirical asset pricing. Fortunately, there is a neat package by [Nelson Areal](https://github.com/nareal/frenchdata/) that allows us to easily access the data

We can use the main function of the package to download monthly Fama-French factors. The set *3 Factors* includes the return time series of the market, size, and value factors alongside the risk-free rates. Note that we have to do some manual work to correctly parse all the columns and scale them appropriately as the raw Fama-French data comes in very unpractical data format. For precise descriptions of the variables, we suggest consulting [Prof. Kenneth French finance data library](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) directly.

In [14]:
import pandas as pd
factors_ff_monthly_raw=pd.read_csv('D:/Tidy/factors_ff_monthly_raw.csv')
startdate = pd.to_datetime('1960-01-01')
enddate = pd.to_datetime('2020-12-31')

In [15]:
factors_ff_monthly_raw['month']=pd.to_datetime(factors_ff_monthly_raw['date'], format='%Y%m')

In [18]:
factors_ff_monthly=pd.DataFrame({'month':factors_ff_monthly_raw['month'],
                                 'rf':factors_ff_monthly_raw['RF']/100,
                                 'mkt_excess':factors_ff_monthly_raw['Mkt-RF']/100,
                                 'smb':factors_ff_monthly_raw['SMB']/100,
                                 'hml':factors_ff_monthly_raw['HML']/100,          })
factors_ff_monthly = factors_ff_monthly.loc[(factors_ff_monthly.month >= startdate) & (factors_ff_monthly.month <= enddate)].reset_index(drop = True)

It is straightforward to download the corresponding *daily* Fama-French factors with the same function. 

In [26]:
factors_ff_daily_raw=pd.read_csv('D:/Tidy/factors_ff_daily_raw.csv')
factors_ff_daily_raw['date']=pd.to_datetime(factors_ff_daily_raw['date'], format='%Y%m%d')

In [27]:
factors_ff_daily=pd.DataFrame({'date':factors_ff_daily_raw['date'],
                                 'rf':factors_ff_daily_raw['RF']/100,
                                 'mkt_excess':factors_ff_daily_raw['Mkt-RF']/100,
                                 'smb':factors_ff_daily_raw['SMB']/100,
                                 'hml':factors_ff_daily_raw['HML']/100,          })
factors_ff_daily = factors_ff_daily.loc[(factors_ff_daily.date >= startdate) & (factors_ff_daily.date <= enddate)].reset_index(drop = True)

In a subsequent chapter, we also use the 10 monthly industry portfolios, so let us fetch that data, too. 

In [39]:
industries_ff_monthly_raw=pd.read_csv('D:/Tidy/industries_ff_monthly_raw.csv')
industries_ff_monthly_raw['month']=pd.to_datetime(factors_ff_monthly_raw['date'], format='%Y%m')

In [41]:
industries_ff_monthly=industries_ff_monthly_raw.loc[:,'NoDur':'Other']/100
industries_ff_monthly['month']=industries_ff_monthly_raw['month']
industries_ff_monthly = industries_ff_monthly.loc[(industries_ff_monthly.month >= startdate) & (industries_ff_monthly.month <= enddate)].reset_index(drop = True)

## q-factors

In recent years, the academic discourse experienced the rise of alternative factor models, e.g., in the form of the [Hou2015](https://doi.org/10.1093/rfs/hhu068) *q*-factor model. We refer to the [extended background](http://global-q.org/background.html) information provided by the original authors for further information. The *q* factors can be downloaded directly from the authors' homepage from within `read_csv()`.

We also need to adjust this data. First, we discard information we will not use here. Then, we rename the columns with the "R_"-prescript using regular expressions and write all column names in lower case.

In [45]:
factors_q_monthly_raw=pd.read_csv("http://global-q.org/uploads/1/2/2/6/122679606/q5_factors_monthly_2020.csv")

In [49]:
from datetime import datetime
factors_q_monthly_raw

,year,month,R_F,R_MKT,R_ME,R_IA,R_ROE,R_EG
0,1967,1,0.3927,8.1852,6.7630,-3.0018,1.9603,-2.2396
1,1967,2,0.3743,0.7557,1.7323,-0.2696,3.5877,2.5681
2,1967,3,0.3693,4.0169,1.9541,-1.6969,1.8805,-1.4695
3,1967,4,0.3344,3.8786,-0.7423,-2.9555,1.0983,-2.2351
4,1967,5,0.3126,-4.2807,2.9255,2.4553,0.5435,0.2749
...,...,...,...,...,...,...,...,...
643,2020,8,0.0079,7.6270,-2.3063,-2.8534,-0.8682,-1.6505
644,2020,9,0.0076,-3.6271,0.3254,-2.1992,1.2255,0.6656
645,2020,10,0.0073,-2.1009,2.7930,-0.7387,-2.4671,-0.8514
646,2020,11,0.0066,12.4689,4.3248,3.0672,-14.4600,-9.7226


In [56]:
factors_q_monthly_raw['month']=factors_q_monthly_raw.apply(lambda x: datetime(int(x.year),int(x.month),1),axis=1)

In [60]:
factors_q_monthly=factors_q_monthly_raw.loc[:,'R_ME':'R_EG']/100
factors_q_monthly['month']=factors_q_monthly_raw['month']
factors_q_monthly=factors_q_monthly.rename(columns={'R_ME':'me',
                                                    'R_IA':'ia',
                                                    'R_ROE':'roe',	
                                                    'R_EG':'eg'})
factors_q_monthly = factors_q_monthly.loc[(factors_q_monthly.month >= startdate) & (factors_q_monthly.month <= enddate)].reset_index(drop = True)

## Macroeconomic predictors

Our next data source is a set of macroeconomic variables often used as predictors for the equity premium. [Goyal2008](https://doi.org/10.1093/rfs/hhm014) comprehensively reexamine the performance of variables suggested by the academic literature to be good predictors of the equity premium. The authors host the data updated to 2020 on [Amit Goyal's website](https://sites.google.com/view/agoyal145). Since the data is a .xlsx-file stored on a public Google drive location, we need additional packages to access the data directly from our R session. Therefore, we load `read_excel()` to read the .xlsx-file.

In [72]:
macro_predictors_raw = pd.read_excel("D:/Tidy/PredictorData2020.xlsx")

d:\Users\81913\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [73]:
macro_predictors_raw['month']=pd.to_datetime(macro_predictors_raw ['yyyymm'], format='%Y%m')
macro_predictors_raw  = macro_predictors_raw .loc[(macro_predictors_raw .month >= startdate) & (macro_predictors_raw .month <= enddate)].reset_index(drop = True)

In [83]:
import numpy as np
macro_predictors_raw['IndexDiv']=macro_predictors_raw['Index'] +macro_predictors_raw['D12']
macro_predictors_raw['logret']=np.log(macro_predictors_raw['IndexDiv']) -np.log(macro_predictors_raw['IndexDiv'].shift(1))
macro_predictors_raw['Rfree']=np.log(macro_predictors_raw['Rfree'] + 1)
macro_predictors_raw['rp_div']=(macro_predictors_raw['logret'] - macro_predictors_raw['Rfree']).shift(-1)
macro_predictors_raw['dp']=np.log(macro_predictors_raw['D12'])-np.log(macro_predictors_raw['Index'])
macro_predictors_raw['dy']=np.log(macro_predictors_raw['D12'])-np.log(macro_predictors_raw['Index'].shift(1))
macro_predictors_raw['ep']=np.log(macro_predictors_raw['E12'])-np.log(macro_predictors_raw['Index'])
macro_predictors_raw['de']=np.log(macro_predictors_raw['D12'])-np.log(macro_predictors_raw['E12'])
macro_predictors_raw['tms']=macro_predictors_raw['lty']-macro_predictors_raw['tbl']
macro_predictors_raw['dfy']=macro_predictors_raw['BAA']-macro_predictors_raw['AAA']

In [88]:
macro_predictors=macro_predictors_raw[['month','rp_div', 'dp', 'dy', 'ep', 'de', 'svar',
         'b/m', 'ntis', 'tbl', 'lty', 'ltr',
         'tms', 'dfy', 'infl']]
macro_predictors=macro_predictors.rename(columns={'b/m':'bm'})
macro_predictors=macro_predictors.dropna()

Finally, after reading in the macro predictors to our memory, we remove the raw data file from our temporary storage. 

## Other macroeconomic data

The Federal Reserve bank of St. Louis provides the Federal Reserve Economic Data (FRED), an extensive database for macroeconomic data. In total, there are 817,000 US and international time series from 108 different sources. As an illustration, we use the already familiar `pandas` package to fetch consumer price index (CPI) data that can be found under the [CPIAUCNS](https://fred.stlouisfed.org/series/CPIAUCNS) key. 

In [108]:
cpi_monthly=pd.read_csv("D:/Tidy/cpi_monthly.csv")

In [109]:
cpi_monthly['month']=pd.to_datetime(cpi_monthly['date'])

In [110]:
cpi_monthly['cpi']=cpi_monthly['price']/float(cpi_monthly.iloc[-1:,:]['price'].values)

In [112]:
cpi_monthly=cpi_monthly.drop(['symbol','date','price','Unnamed: 0'],axis=1)

## Setting up a database

Now that we have downloaded some data from the web into the memory of our Python session, let us set up a database to store that information for future use. We will use the data stored in this database throughout the following chapters, but you could alternatively implement a different strategy and replace the respective code. 

There are many ways to set up and organize a database, depending on the use case. For our purpose, the most efficient way is to use an [SQLite](https://www.sqlite.org/index.html) database, which is the C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. Note that [SQL](https://en.wikipedia.org/wiki/SQL) (Structured Query Language) is a standard language for accessing and manipulating databases. We refer to [this tutorial](https://www.w3schools.com/sql/sql_intro.asp) for more information on SQL. 

There are two packages that make working with SQLite in R very simple: `sqlite3` embeds the SQLite database engine in Python. These packages allow to set up a database to remotely store tables and use these remote database tables. Check out the [sqlite3](https://docs.python.org/3/library/sqlite3.html).

In [113]:
import sqlite3
tidy_finance = sqlite3.connect('D:/Tidy/tidyfinance.sqlite')

In [114]:
factors_ff_monthly.to_sql('factors_ff_monthly',tidy_finance,if_exists='replace')

In [115]:
factors_ff_daily.to_sql('factors_ff_daily',tidy_finance,if_exists='replace')

In [116]:
industries_ff_monthly.to_sql('industries_ff_monthly',tidy_finance,if_exists='replace')

factors_q_monthly.to_sql('factors_q_monthly',tidy_finance,if_exists='replace')

macro_predictors.to_sql('macro_predictors',tidy_finance,if_exists='replace')

cpi_monthly.to_sql('cpi_monthly',tidy_finance,if_exists='replace')